<a href="https://colab.research.google.com/github/xuanthang482/CS114.K21/blob/master/sarcasm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

lấy data từ kaggle !

In [2]:
from google.colab import files
files.upload()


Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"hongxunthng","key":"873be95c35568ebb1940a27daed561a5"}'}

In [3]:
!ls -lha kaggle.json
!pip install -q kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

-rw-r--r-- 1 root root 67 May  8 08:47 kaggle.json


In [4]:
!kaggle datasets download -d rmisra/news-headlines-dataset-for-sarcasm-detection

  0% 0.00/3.30M [00:00<?, ?B/s]
100% 3.30M/3.30M [00:00<00:00, 111MB/s]


In [5]:
!unzip news-headlines-dataset-for-sarcasm-detection

Archive:  news-headlines-dataset-for-sarcasm-detection.zip
  inflating: Sarcasm_Headlines_Dataset.json  
  inflating: Sarcasm_Headlines_Dataset_v2.json  


new section 

In [0]:
import pandas as pd, numpy as np, re, time
from nltk.stem.porter import PorterStemmer

In [0]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

In [0]:
data = pd.read_json('Sarcasm_Headlines_Dataset.json', lines = True)


In [10]:
data.head()

,article_link,headline,is_sarcastic
0,https://www.huffingtonpost.com/entry/versace-b...,former versace store clerk sues over secret 'b...,0
1,https://www.huffingtonpost.com/entry/roseanne-...,the 'roseanne' revival catches up to our thorn...,0
2,https://local.theonion.com/mom-starting-to-fea...,mom starting to fear son's web series closest ...,1
3,https://politics.theonion.com/boehner-just-wan...,"boehner just wants wife to listen, not come up...",1
4,https://www.huffingtonpost.com/entry/jk-rowlin...,j.k. rowling wishes snape happy birthday in th...,0


check the null value

In [11]:
print(data.isnull().any(axis = 0))

article_link    False
headline        False
is_sarcastic    False
dtype: bool


XỬ LÍ DATA

Ở cột headline sẽ có 1 vài kí tự cần được loại bỏ, dùng regular expression để loại bỏ các kí tự này

In [0]:
data['headline'] = data['headline'].apply(lambda s : re.sub('[^a-zA-Z]', ' ', s))



Gán feature và label

In [0]:
features = data['headline']
labels = data['is_sarcastic']


Steaming data

In [0]:
ps = PorterStemmer()
features = features.apply(lambda x: x.split())
features = features.apply(lambda x : ' '.join([ps.stem(word) for word in x]))

Vector hóa các tính năng bằng cách sử dụng TF-IDF Vectorizer 

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer
tv = TfidfVectorizer(max_features = 5000)
features = list(features)
features = tv.fit_transform(features).toarray()

TRAIN AND TEST DATA

getting training and testing data

In [0]:
features_train, features_test, labels_train, labels_test = train_test_split(features, labels, test_size = .05, random_state = 0)

model 1: Using linear support vector classifier

In [17]:
lsvc = LinearSVC()
# training the model
lsvc.fit(features_train, labels_train)
# getting the score of train and test data
print('train:', lsvc.score(features_train, labels_train)) 
print('test:', lsvc.score(features_test, labels_test))

train: 0.9093524612777362
test: 0.8375748502994012
